<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/Ejercicio_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 6

Clasificación con Naive Bayes

In [120]:
import pandas as pd
import numpy as np
     

import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d muhammadtalharasool/simple-gender-classification

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
simple-gender-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [121]:
# Extraigo todos los archivos que están en el .zip que acabamos de "bajar". 

import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [158]:
df = pd.read_csv('/content/gender.csv')

In [159]:
df

,Gender,Age,Height (cm),Weight (kg),Occupation,Education Level,Marital Status,Income (USD),Favorite Color,Unnamed: 9
0,male,32,175,70,Software Engineer,Master's Degree,Married,75000,Blue,NaN
1,male,25,182,85,Sales Representative,Bachelor's Degree,Single,45000,Green,NaN
2,female,41,160,62,Doctor,Doctorate Degree,Married,120000,Purple,NaN
3,male,38,178,79,Lawyer,Bachelor's Degree,Single,90000,Red,NaN
4,female,29,165,58,Graphic Designer,Associate's Degree,Single,35000,Yellow,NaN
...,...,...,...,...,...,...,...,...,...,...
126,female,32,170,64,Nurse,Associate's Degree,Single,60000,Orange,NaN
127,male,38,176,79,Project Manager,Bachelor's Degree,Married,90000,Black,NaN
128,female,27,162,55,Graphic Designer,Associate's Degree,Single,55000,Green,NaN
129,male,33,175,77,Sales Representative,Bachelor's Degree,Married,80000,Yellow,NaN


In [160]:
# Los nombres de las columnas tienen un problema más común que lo deseable: espacios escondidos
df.columns

Index([' Gender', ' Age', ' Height (cm)', ' Weight (kg)', ' Occupation',
       ' Education Level', ' Marital Status', ' Income (USD)',
       ' Favorite Color', 'Unnamed: 9'],
      dtype='object')

In [161]:
# Una solución posible es eliminarlos todos
df.columns = df.columns.str.replace(' ', '')

In [162]:
df.columns

Index(['Gender', 'Age', 'Height(cm)', 'Weight(kg)', 'Occupation',
       'EducationLevel', 'MaritalStatus', 'Income(USD)', 'FavoriteColor',
       'Unnamed:9'],
      dtype='object')

In [163]:
# Pero esos espacios escondidos también aparecen en otros lados. Hay que tener cuidado. 
df.Gender.value_counts()

male       41
female     39
 male      27
 female    24
Name: Gender, dtype: int64

In [164]:
df['Gender'] = df['Gender'].str.replace(' ', '')

In [165]:
df.Gender.value_counts()

male      68
female    63
Name: Gender, dtype: int64

In [166]:
# Lo hacemos para todas las categóricas
df['Occupation'] = df['Occupation'].str.replace(' ', '')
df['EducationLevel'] = df['EducationLevel'].str.replace(' ', '')
df['MaritalStatus'] = df['MaritalStatus'].str.replace(' ', '')
df['FavoriteColor'] = df['FavoriteColor'].str.replace(' ', '')

In [167]:
# Hay una columna que está mal, así que la podemos borrar también
df=df.drop(['Unnamed:9'], axis=1)

In [174]:
cat=['Occupation','EducationLevel', 'MaritalStatus', 'FavoriteColor']
for c in cat: 
  df=df.join(pd.get_dummies(df[c], dummy_na=False))

In [175]:
df

,Gender,Age,Height(cm),Weight(kg),Occupation,EducationLevel,MaritalStatus,Income(USD),FavoriteColor,Accountant,...,Widowed,Black,Blue,Green,Grey,Orange,Pink,Purple,Red,Yellow
0,male,32,175,70,SoftwareEngineer,Master'sDegree,Married,75000,Blue,0,...,0,0,1,0,0,0,0,0,0,0
1,male,25,182,85,SalesRepresentative,Bachelor'sDegree,Single,45000,Green,0,...,0,0,0,1,0,0,0,0,0,0
2,female,41,160,62,Doctor,DoctorateDegree,Married,120000,Purple,0,...,0,0,0,0,0,0,0,1,0,0
3,male,38,178,79,Lawyer,Bachelor'sDegree,Single,90000,Red,0,...,0,0,0,0,0,0,0,0,1,0
4,female,29,165,58,GraphicDesigner,Associate'sDegree,Single,35000,Yellow,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,female,32,170,64,Nurse,Associate'sDegree,Single,60000,Orange,0,...,0,0,0,0,0,1,0,0,0,0
127,male,38,176,79,ProjectManager,Bachelor'sDegree,Married,90000,Black,0,...,0,1,0,0,0,0,0,0,0,0
128,female,27,162,55,GraphicDesigner,Associate'sDegree,Single,55000,Green,0,...,0,0,0,1,0,0,0,0,0,0
129,male,33,175,77,SalesRepresentative,Bachelor'sDegree,Married,80000,Yellow,0,...,0,0,0,0,0,0,0,0,0,1


In [176]:
from sklearn.model_selection import train_test_split
X=list(df.columns)
X.remove('Gender')
X_train, X_test, y_train, y_test = train_test_split(df[X], df['Gender'], test_size=0.2, random_state=0)

In [ ]:
for c in cat: X.remove(c)

In [180]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train[X], y_train)

MultinomialNB()

In [182]:
clf.score(X_train[X], y_train)

0.7980769230769231

In [183]:
clf.score(X_test[X], y_test)

0.7777777777777778

In [184]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train[X], y_train)

GaussianNB()

In [185]:
clf.score(X_train[X], y_train)

1.0

In [186]:
clf.score(X_test[X], y_test)

1.0

In [187]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train[X], y_train)

LogisticRegression()

In [188]:
clf.score(X_train[X], y_train)

1.0

In [189]:
clf.score(X_test[X], y_test)

1.0